# Import learning data

In [1]:
import pandas as pd

training_data1_path = "../../learning_data/data/win_rate/river/50000-data-1000-simulation-2-players-win-rate-data.csv"
training_data2_path = "../../learning_data/data/win_rate/river/100000-data-1000-simulation-2-players-win-rate-data.csv"
test_data_path = "../../learning_data/data/win_rate/river/10000-data-1000-simulation-2-players-win-rate-data.csv"
train1_df = pd.read_csv(training_data1_path)
train2_df = pd.read_csv(training_data2_path)
train_df = train1_df.append(train2_df, ignore_index=True)
test_df = pd.read_csv(test_data_path)

# About learning data

In [2]:
print train_df.shape, test_df.shape

(150000, 15) (10000, 15)


In [3]:
train_df.head()

,hole1,hole2,community1,community2,community3,community4,community5,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id,community4_id,community5_id
0,S2,C8,C5,C9,DA,SA,S5,0.812,41,8,5,9,14,40,44
1,CA,D2,DJ,D6,H8,SK,CJ,0.516,1,15,24,19,34,52,11
2,S6,S8,SA,C9,HA,H6,C8,0.555,45,47,40,9,27,32,8
3,CK,SQ,C6,ST,SK,HA,C5,0.794,13,51,6,49,52,27,5
4,SQ,H5,ST,S9,SA,D9,H9,0.600,51,31,49,48,40,22,35


In [4]:
train_df.describe()

,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id,community4_id,community5_id
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.586481,26.439747,26.474587,26.436660,26.553287,26.515253,26.546753,26.517560
std,0.264026,15.002405,14.966971,15.025015,14.981690,15.014688,15.011183,15.021035
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.456000,13.000000,14.000000,13.000000,14.000000,13.000000,14.000000,14.000000
50%,0.568000,26.000000,26.000000,26.000000,27.000000,27.000000,27.000000,26.500000
75%,0.813000,39.000000,39.000000,40.000000,40.000000,40.000000,40.000000,40.000000
max,1.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000


In [5]:
test_df.describe()

,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id,community4_id,community5_id
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.584548,26.785300,26.192400,26.672400,26.400100,26.506900,26.501400,26.665500
std,0.264149,14.959572,15.023706,14.896425,15.025145,14.938101,15.103715,14.998703
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.456000,14.000000,13.000000,14.000000,13.000000,14.000000,13.000000,14.000000
50%,0.565000,27.000000,26.000000,27.000000,26.000000,26.000000,27.000000,27.000000
75%,0.811000,40.000000,39.000000,40.000000,39.000000,39.000000,40.000000,40.000000
max,1.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000


# Data Processing

## card id -> 1-hot vector

In [6]:
import numpy as np
from pypokerengine.engine.card import Card

fetch_hole = lambda row: [row[key] for key in ['hole1_id', 'hole2_id']]
fetch_community = lambda row: [row[key] for key in ['community1_id', 'community2_id', 'community3_id', 'community4_id', 'community5_id']]
gen_card_rank_vec = lambda card: [1 if r == card.rank else 0 for r in range(2,15)]
gen_card_suit_vec = lambda card: [1 if s == card.suit else 0 for s in [Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE]]
gen_card_vec = lambda card: gen_card_rank_vec(card) + gen_card_suit_vec(card)
gen_img = lambda zipped: [gen_card_vec(Card.from_id(card_id)) for card_id in zipped[0]+zipped[1]]

train_hole = train_df.apply(lambda row: fetch_hole(row), axis=1)
train_community = train_df.apply(lambda row: fetch_community(row), axis=1)
train_df["onehot"] = map(gen_img, zip(train_hole, train_community))

test_hole = test_df.apply(lambda row: fetch_hole(row), axis=1)
test_community = test_df.apply(lambda row: fetch_community(row), axis=1)
test_df["onehot"] = map(gen_img, zip(test_hole, test_community))

## Format data (pandas.df -> numpy.ndarray)

In [7]:
wrap = lambda lst: np.array(lst)
to_ndarray = lambda X: wrap([wrap(x) for x in X])
train_x, train_y = [to_ndarray(array) for array in [train_df["onehot"].values, train_df["win_rate"].values]]
test_x, test_y = [to_ndarray(array) for array in [test_df["onehot"].values, test_df["win_rate"].values]]
train_x, test_x = [X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) for X in [train_x, test_x]]
print "shape of training data => x: %s, y: %s" % (train_x.shape, train_y.shape)
print "shape of test data => x: %s, y: %s" % (test_x.shape, test_y.shape)

shape of training data => x: (150000, 1, 7, 17), y: (150000,)
shape of test data => x: (10000, 1, 7, 17), y: (10000,)


# Create model

In [8]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dense, Activation, Dropout, Flatten

model = Sequential()

model.add(Convolution2D(256, 3, 3, border_mode="same", input_shape=(1, 7, 17)))
model.add(Activation("relu"))
model.add(Convolution2D(256, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(Convolution2D(256, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss="mse",  optimizer="adadelta")

Using Theano backend.


# Train or Load existing model

In [9]:
import os
model_weight_file_path = "./resource/river_cnn_model_weights.h5"
if os.path.exists(model_weight_file_path):
    model.load_weights(model_weight_file_path)
else:
    history = model.fit(train_x, train_y, batch_size=128, nb_epoch=1, validation_split=0.1, verbose=2)

# Check model performance

## Visualize loss transition

In [10]:
%matplotlib inline
import seaborn
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for idx, key in enumerate(history.history, start=1):
    plt.subplot(2, 1, idx)
    plt.plot(range(len(history.history[key])), history.history[key])
    plt.title(key)
plt.show()

<img src="./resource/river_cnn_loss_history.png" width=800/>

## Test model performance by MSE 

In [10]:
from sklearn.metrics import mean_squared_error

def print_model_performance(model, train_x, train_y, test_x, test_y):
    print 'MSE on training data = {score}'.format(score=mean_squared_error(model.predict(train_x), train_y))
    print 'MSE on test data = {score}'.format(score=mean_squared_error(model.predict(test_x), test_y))

In [11]:
print_model_performance(model, train_x, train_y, test_x, test_y)

MSE on training data = 0.000217936295025
MSE on test data = 0.000452377389787


## See model prediction on sample data

In [12]:
from pypokerengine.engine.card import Card
C, D, H, S = Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE

test_case = [
    [(11, H), (7, S), (12, H), (6, S), (1, H), (10, H), (13, H), 1.0],
    [(1, S), (4, H), (2, S), (4, S), (4, D), (1, H), (5, H), 0.993],
    [(8, D), (11, C), (3, S), (6, H), (10, C), (5, C), (4, D), 0.018],
    [(2, D), (13, S), (9, D), (5, C), (5, D), (1, H), (4, H), 0.501]
]

to_id = lambda card: card.to_id()
gen_card_rank_vec = lambda card: [1 if r == card.rank else 0 for r in range(2,15)]
gen_card_suit_vec = lambda card: [1 if s == card.suit else 0 for s in [Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE]]
gen_card_vec = lambda card: gen_card_rank_vec(card) + gen_card_suit_vec(card)
gen_img = lambda zipped: [gen_card_vec(Card.from_id(card_id)) for card_id in zipped[0]+zipped[1]]
wrap = lambda lst: np.array(lst)
to_ndarray = lambda X: wrap([wrap(x) for x in X])

for card1, card2, card3, card4, card5, card6, card7, expected in test_case:
    cards = [Card(rank=rank, suit=suit) for rank, suit in [card1, card2, card3, card4, card5, card6, card7]]
    hole = cards[:2]
    community = cards[2:]
    hole_ids = map(to_id, hole)
    community_ids = map(to_id, community)
    x = gen_img((hole_ids, community_ids))
    X = to_ndarray(x)
    X = np.array([X.reshape(1, X.shape[0], X.shape[1])])
    y = model.predict(X)[0][0]
    print "HOLE = [%s, %s], COMMUNITY = [%s, %s, %s, %s, %s] => win_rate = { prediction=%f, expected=%f }" % tuple(map(str, hole) + map(str, community) + [y , expected])

HOLE = [HJ, S7], COMMUNITY = [HQ, S6, HA, HT, HK] => win_rate = { prediction=0.985628, expected=1.000000 }
HOLE = [SA, H4], COMMUNITY = [S2, S4, D4, HA, H5] => win_rate = { prediction=0.974373, expected=0.993000 }
HOLE = [D8, CJ], COMMUNITY = [S3, H6, CT, C5, D4] => win_rate = { prediction=0.029780, expected=0.018000 }
HOLE = [D2, SK], COMMUNITY = [D9, C5, D5, HA, H4] => win_rate = { prediction=0.514406, expected=0.501000 }
